### Imports 

In [25]:
import pandas as pd 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from itertools import combinations


### Load Data

In [185]:
!pipenv install xlsxwriter

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing xlsxwriter...
Adding xlsxwriter to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock (abeb6c) out of date, updating to (e6e9c1)...
Locking [dev-packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (e6e9c1)!
Installing dependencies from Pipfile.lock (e6e9c1)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 0/0 — 00:00:00
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [141]:
## sample data 
POLAND_DATA = "./data/Poland - PL.xlsx"
POLAND_SHEET_NAME = "Relevant KW List (10,940)"

## load data
df = pd.read_excel(POLAND_DATA, sheet_name=POLAND_SHEET_NAME)

## keywords column 
kw_col = "Keyword - PL"

msv_col = "Average MSV"

## get keywords
keywords = df[kw_col].tolist()

## get the unique keywords
unique_keywords = list(set(keywords))

## print number of original keywords 
print("Number of original keywords: ", len(keywords))

## print the number of unique keywords
print("Number of unique keywords: ", len(unique_keywords))

Number of original keywords:  10940
Number of unique keywords:  10940


In [183]:
pd.read_excel(POLAND_DATA, sheet_name = 1)

,#,Keyword - PL,Keyword - EN,Average MSV,B/N,Brand,Business Type,Topic,Sub-Topic
0,NaN,klimatyzator,air conditioner,60500,NonBranded,-,NaN,Information,Basic-Knowledge
1,NaN,klimatyzator przenośny,portable air conditioner,60500,NonBranded,-,NaN,Type,Portable
2,NaN,klimatyzatory,air conditioners,60500,NonBranded,-,NaN,Information,Basic-Knowledge
3,NaN,klimatyzator przenosny,portable air conditioner,60500,NonBranded,-,NaN,Type,Portable
4,NaN,klimatyzacja,air conditioning,49500,NonBranded,-,NaN,Information,Basic-Knowledge
...,...,...,...,...,...,...,...,...,...
10935,NaN,wentylatory do klimatyzacji,air conditioning fans,10,NonBranded,-,NaN,Type,Air-Fan
10936,NaN,wentylator zamiast klimatyzacji,fan instead of air conditioning,10,NonBranded,-,NaN,Type,Air-Fan
10937,NaN,klimatyzacja bez wentylatora,air conditioning without fan,10,NonBranded,-,NaN,Type,Air-Fan
10938,NaN,system klimatyzacji vrv,vrv air conditioning system,10,NonBranded,-,NaN,System,VRF-/-VRV


In [154]:
def get_fuzzy_match_score(a: str, b: str) -> int:
    """
    Get the fuzzy match score between two strings
    """
    return fuzz.ratio(a, b)

In [195]:
## another approach: 
TOP_THRESHOLD = 99
BOTTOM_THRESHOLD = 40

## groupby Average MSV and join the keywords 
df_grouped = df.groupby(msv_col)[kw_col].apply(lambda x: ', '.join(x)).reset_index()

## for each keyword in the keywords column, use the first keyword as the pivot, and calculate the fuzzy match score with the rest of the keywords
## then remove the ones with a score of 100, and one below 60 
## create a new column to hold the keywords that are similar to the pivot keyword, joined by a "/"
df_grouped['similar_keywords'] = df_grouped[kw_col].apply(lambda x: '/'.join([kw for kw in x.split(',') if get_fuzzy_match_score(x.split(',')[0], kw) > BOTTOM_THRESHOLD and get_fuzzy_match_score(x.split(',')[0], kw) < TOP_THRESHOLD]))

## add the similar_keywords column to the original dataframe, by matching the Average MSV 
df_merged = df.merge(df_grouped[[msv_col, 'similar_keywords']], on=msv_col, how='left')

## save it to an excel file 
# df_merged.to_excel("PL - POLAND - similar_keywords.xlsx", index=False)

In [198]:
## if the keyword in the kw_col is in the similar_keywords col, then remove it from the similar_keywords col
df_two = df_merged.copy()
## iterate over the rows in the dataframe
for index, row in df_two.iterrows():
    ## if the keyword in the kw_col is in the similar_keywords col, then remove it from the similar_keywords col
    if row[kw_col] in row['similar_keywords']:
        ## split the similar_keywords col by "/"
        similar_keywords = row['similar_keywords'].split('/')
        ## remove the keyword from the similar_keywords col
        if row[kw_col] in similar_keywords:
            similar_keywords.remove(row[kw_col])
        ## join the similar_keywords col by "/"
        df_two.at[index, 'similar_keywords'] = '/'.join(similar_keywords)


In [199]:
df_two

,#,Keyword - PL,Keyword - EN,Average MSV,B/N,Brand,Business Type,Topic,Sub-Topic,similar_keywords
0,NaN,klimatyzator,air conditioner,60500,NonBranded,-,NaN,Information,Basic-Knowledge,klimatyzator przenośny/ klimatyzatory/ klimat...
1,NaN,klimatyzator przenośny,portable air conditioner,60500,NonBranded,-,NaN,Type,Portable,klimatyzator przenośny/ klimatyzatory/ klimat...
2,NaN,klimatyzatory,air conditioners,60500,NonBranded,-,NaN,Information,Basic-Knowledge,klimatyzator przenośny/ klimatyzatory/ klimat...
3,NaN,klimatyzator przenosny,portable air conditioner,60500,NonBranded,-,NaN,Type,Portable,klimatyzator przenośny/ klimatyzatory/ klimat...
4,NaN,klimatyzacja,air conditioning,49500,NonBranded,-,NaN,Information,Basic-Knowledge,klimatyzacji
...,...,...,...,...,...,...,...,...,...,...
10935,NaN,wentylatory do klimatyzacji,air conditioning fans,10,NonBranded,-,NaN,Type,Air-Fan,mala klima/ multisplit klima/ koszt nabicia k...
10936,NaN,wentylator zamiast klimatyzacji,fan instead of air conditioning,10,NonBranded,-,NaN,Type,Air-Fan,mala klima/ multisplit klima/ koszt nabicia k...
10937,NaN,klimatyzacja bez wentylatora,air conditioning without fan,10,NonBranded,-,NaN,Type,Air-Fan,mala klima/ multisplit klima/ koszt nabicia k...
10938,NaN,system klimatyzacji vrv,vrv air conditioning system,10,NonBranded,-,NaN,System,VRF-/-VRV,mala klima/ multisplit klima/ koszt nabicia k...


,#,Keyword - PL,Keyword - EN,Average MSV,B/N,Brand,Business Type,Topic,Sub-Topic,similar_keywords
0,NaN,klimatyzator,air conditioner,60500,NonBranded,-,NaN,Information,Basic-Knowledge,przenośny/ y/ przenosny
1,NaN,klimatyzator przenośny,portable air conditioner,60500,NonBranded,-,NaN,Type,Portable,/ klimatyzatory/ klimatyzator przenosny
2,NaN,klimatyzatory,air conditioners,60500,NonBranded,-,NaN,Information,Basic-Knowledge,klimatyzator przenośny/ / klimatyzator przenosny
3,NaN,klimatyzator przenosny,portable air conditioner,60500,NonBranded,-,NaN,Type,Portable,klimatyzator przenośny/ klimatyzatory/
4,NaN,klimatyzacja,air conditioning,49500,NonBranded,-,NaN,Information,Basic-Knowledge,klimatyzacji
...,...,...,...,...,...,...,...,...,...,...
10935,NaN,wentylatory do klimatyzacji,air conditioning fans,10,NonBranded,-,NaN,Type,Air-Fan,mala klima/ multisplit klima/ koszt nabicia k...
10936,NaN,wentylator zamiast klimatyzacji,fan instead of air conditioning,10,NonBranded,-,NaN,Type,Air-Fan,mala klima/ multisplit klima/ koszt nabicia k...
10937,NaN,klimatyzacja bez wentylatora,air conditioning without fan,10,NonBranded,-,NaN,Type,Air-Fan,mala klima/ multisplit klima/ koszt nabicia k...
10938,NaN,system klimatyzacji vrv,vrv air conditioning system,10,NonBranded,-,NaN,System,VRF-/-VRV,mala klima/ multisplit klima/ koszt nabicia k...
